In [1]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |██████████████████████████████  | 1915.4 MB 58.4 MB/s eta 0:00:03     |████████████████▌               | 1054.4 MB 59.5 MB/s eta 0:00:17     |██████████████████▉             | 1201.5 MB 65.2 MB/s eta 0:00:13     |█████████████████████▌          | 1372.4 MB 78.6 MB/s eta 0:00:09

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 2041.3 MB 114.4 MB/s eta 0:00:01     |████████████████████████████████| 2041.3 MB 15 kB/s 
     |████████████████████████████████| 23.2 MB 9.8 MB/s eta 0:00:01
  Using cached torchaudio-0.9.0-cp37-cp37m-manylinux1_x86_64.whl (1.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1+cu110
    Uninstalling torch-1.7.1+cu110:
      Successfully uninstalled torch-1.7.1+cu110
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0
    Uninstalling torchaudio-0.11.0:
      Successfully uninstalled torchaudio-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 1.1.0 requires torch<1.7.0,>=1

In [3]:
wandb login

SyntaxError: invalid syntax (118173973.py, line 1)

In [4]:
wandb.init(project='my_first_test')

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/guneetsk99/.netrc


In [5]:
import torch
import numpy as np
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from datasets import Dataset

# Load the model
tokenizer = AutoTokenizer.from_pretrained("johngiorgi/declutr-base")
model = AutoModel.from_pretrained("johngiorgi/declutr-base" ).to('cuda')

# Prepare some text to embed
dataset = load_dataset("tomekkorbak/pile-nontoxic-chunk-0")
train_dataset = Dataset.from_dict(dataset['train'].to_dict())
# small dataset with 1000 samples
small_dataset = train_dataset.select([i for i in range(9000)])

Some weights of the model checkpoint at johngiorgi/declutr-base were not used when initializing RobertaModel: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration tomekkorbak--pile-nontoxic-chunk-0-52be9de9481b387c
Reusing dataset parquet (/home/guneetsk99/.cache/huggingface/datasets/tomekkorbak___parquet/tomekkorbak--pile-nontoxic-chunk-0-52be9de9481b387c/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:

def mean_pooling(model_output, sample):
    return torch.sum(
        model_output * sample["attention_mask"].unsqueeze(-1), dim=1
        ) / torch.clamp(torch.sum(sample["attention_mask"], dim=1, keepdims=True), min=1e-9)
        
def get_embeddings(examples):
    text_list = examples['text']
    encoded_input = tokenizer(
        text_list, padding='max_length', truncation=True, return_tensors="pt", max_length=512
    ).to('cuda')
    model_output = model(**encoded_input)[0]
    return mean_pooling(model_output, encoded_input)


In [7]:
embeddings_dataset = small_dataset.map(
    lambda x: {"embeddings": get_embeddings(x).cpu().detach().numpy()[0]},
)

  0%|          | 0/9000 [00:00<?, ?ex/s]

In [8]:
import numpy
np_arr=numpy.array(embeddings_dataset['embeddings'])

In [9]:
from autofaiss import build_index
index, index_infos = build_index(np_arr, save_on_disk=False)

2022-06-18 18:35:20,759 [INFO]: Using 96 omp threads (processes), consider increasing --nb_cores if you have more
2022-06-18 18:35:20,798 [INFO]: Launching the whole pipeline 06/18/2022, 18:35:20
2022-06-18 18:35:20,798 [INFO]: Reading total number of vectors and dimension 06/18/2022, 18:35:20
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 14716.86it/s]
2022-06-18 18:35:20,939 [INFO]: There are 9000 embeddings of dim 768
2022-06-18 18:35:20,939 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.1396 secs
2022-06-18 18:35:20,940 [INFO]: 	Compute estimated construction time of the index 06/18/2022, 18:35:20
2022-06-18 18:35:20,940 [INFO]: 		-> Train: 16.7 minutes
2022-06-18 18:35:20,941 [INFO]: 		-> Add: 0.1 seconds
2022-06-18 18:35:20,941 [INFO]: 		Total: 16.7 minutes
2022-06-18 18:35:20,942 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0013 secs
2022-06-18 18:35:20,942 [INFO]: 	Checking that your have enough mem